In [ ]:
curr_dir = '/home/vayzenbe/GitHub_Repos/GiNN'

import sys
sys.path.insert(1, f'{curr_dir}/Models')
import os, argparse
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image, ImageOps,  ImageFilter
from itertools import chain
import pandas as pd
import numpy as np
import cornet
import model_funcs
import matplotlib.pyplot as plt
from statistics import mean
%matplotlib inline
import LoadFrames
from scipy import stats, spatial
from sklearn.decomposition import PCA
from scipy.stats import gamma

In [ ]:
'''
folder params
'''

stim_dir = f"{curr_dir}/Stim/fmri_videos/frames"
weights_dir = f"/lab_data/behrmannlab/vlad/ginn/model_weights"
results_dir = f"{curr_dir}/Results/fmri_ts"


'''
set model params
'''
model_type = 'classify'
train_cond = ['mixed_imagenet_vggface']
n_classes = [1200, 600]
layer =['aIT','pIT'] #set in descending order
epochs = [0, 1, 5, 10, 15, 20, 25, 30]

vid = "partly_cloudy"

transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])])

#If true, load previousl extracted acts; else extract from video
load_acts = True 

'''
fMRI and video parameters
'''

vols = 168 #volumes in the scan
tr = 2 #TR of scan
fix_tr =5 #first 5 volumes of the scan (10 s) were fix in the beginning
fps = 24 # frame per second of video (how many rows go into 1 sec)
bin_size = fps * tr # get the bin size to average by multiplying the FPS by tr



In [ ]:
def model_loop(model, loader):

    im, label = next(iter(loader))

    first_batch = True
    for im, label in loader:
        out = model_funcs.extract_acts(model, im)

        if first_batch == True:
            all_out = out
            first_batch = False
        else:
            all_out = torch.cat((all_out, out), dim=0)

    frame_acts = all_out.cpu().detach().numpy()
    
    return frame_acts

#idx = np.argwhere(np.all(all_out[..., :] == 0, axis=0))# find columns with only zeros
#all_out = np.delete(all_out, idx, axis=1)
#frame_ts = (all_out - np.mean(all_out,axis = 0))/np.std(all_out, axis = 0) # standardize the unit activations; this produces NaNs right now; figure out why
#frame_ts = frame_ts[:, ~np.isnan(frame_ts).any(axis=0)]


In [ ]:
def down_sample(data):
    """Downsample data"""
    downsample_ts = np.empty((0, data.shape[1])) 
    
    #Bin frame data for TS
    for nn in range(0,len(data),bin_size):
        temp = data[nn:(nn+bin_size),:]

        downsample_ts = np.vstack((downsample_ts,np.mean(temp, axis=0)))

    downsample_ts = downsample_ts[0:(vols-fix_tr),:] #extract only 168 volumes to match fmri data (credits of movie were cut)
    
    return downsample_ts

In [ ]:
def extract_pc(data, n_components=None):

    """
    Extract principal components
    if n_components isn't set, it will extract all it can
    
    """
    pca = PCA(n_components = n_components)
    pca.fit(data)
    
    return pca

In [ ]:
def calc_pc_n(pca, thresh):
    '''
    Calculate how many PCs are needed to explain X% of data
    
    pca - result of pca analysis
    thresh- threshold for how many components to keep
    '''

    explained_variance = pca.explained_variance_ratio_
    
    var = 0
    for n_comp, ev in enumerate(explained_variance):
        var += ev #add each PC's variance to current variance
        #print(evn, ev, var)

        if var >=thresh: #once variance > than thresh, stop
            break
    
    #plt.bar(range(len(explained_variance[0:n_comp])), explained_variance[0:n_comp], alpha=0.5, align='center')
    #plt.ylabel('Variance ratio')
    #plt.xlabel('Principal components')
    #plt.show()
    
    return n_comp

In [ ]:
'''
Convolve with HRF using
Using double-gamma with 4-sec peak
'''

def hrf(data, tr):
    """ Return values for HRF at given times """
    times = np.arange(0, 30, tr)
    # Gamma pdf for the peak
    peak_values = gamma.pdf(times, 6)
    # Gamma pdf for the undershoot
    undershoot_values = gamma.pdf(times, 12)
    # Combine them
    values = peak_values - 0.35 * undershoot_values
    # Scale max to 0.6
    hrf_at_trs = values / np.max(values) * 0.6 
    
    
    return np.convolve(data, hrf_at_trs)

In [ ]:
def convolve_hrf(data):

    conv_ts =np.zeros((data.shape))
    for ii in range(0,data.shape[1]):
        temp = hrf(data[:,ii],tr)
        temp = temp[0:(vols-fix_tr)] # only grab the first 168 volumes
        temp = (temp - np.mean(temp))/np.std(temp)
        conv_ts[:,ii] = temp
        
        return conv_ts

In [9]:
dataset = LoadFrames.LoadFrames(f'{stim_dir}/{vid}',  transform=transform)
loader = torch.utils.data.DataLoader(dataset, batch_size=135, shuffle=False,num_workers = 4, pin_memory=True)


In [10]:
'''
Model loop ver. 1
Conducts PCA on down-sample data
'''

for nc, tc in enumerate(train_cond):
    for ee in epochs:
        
        model = model_funcs.load_model(model_type, tc,ee, weights_dir, n_classes[nc])
        
        for ll in layer:
            model =  model_funcs.remove_layer(model, ll)

            #extract acts for all frames of video
            frame_acts = model_loop(model, loader)
            np.save(f'fmri_data/{vid}_{model_type}_{tc}_{ee}_{ll}_allframes', frame_acts)

            #downsample to scale of fmri
            downsample_ts = down_sample(frame_acts)

            #calculate components for pca
            n_comp = calc_pc_n(extract_pc(downsample_ts),.95)
            
            #calculate final set of PCs
            pca2 = extract_pc(downsample_ts, n_comp)
            model_pcs = pca2.transform(downsample_ts) #reduce dimensionality of data using model PCs
            #plot pc variance explained

            #convolve to hrf
            final_ts = convolve_hrf(model_pcs)
            np.save(f'{results_dir}/{vid}_{model_type}_{tc}_{ee}_{ll}_TS', final_ts)
            print(tc, ee,ll, n_comp)

        
    
    
    

mixed_imagenet_vggface 0 aIT 42
mixed_imagenet_vggface 0 pIT 54
mixed_imagenet_vggface 1 aIT 84
mixed_imagenet_vggface 1 pIT 98
mixed_imagenet_vggface 5 aIT 98
mixed_imagenet_vggface 5 pIT 106
mixed_imagenet_vggface 10 aIT 103
mixed_imagenet_vggface 10 pIT 108
mixed_imagenet_vggface 15 aIT 102
mixed_imagenet_vggface 15 pIT 107
mixed_imagenet_vggface 20 aIT 102
mixed_imagenet_vggface 20 pIT 107
mixed_imagenet_vggface 25 aIT 101
mixed_imagenet_vggface 25 pIT 107
mixed_imagenet_vggface 30 aIT 101
mixed_imagenet_vggface 30 pIT 107


In [ ]:
'''
Model loop ver. 2
Conducts PCA on all model data
'''

for nc, tc in enumerate(train_cond):
    for ee in epochs:
        
        model = model_funcs.load_model(model_type, tc,ee, weights_dir, n_classes[nc])
        
        for ll in layer:
            model =  model_funcs.remove_layer(model, ll)

            #extract acts for all frames of video
            frame_acts = model_loop(model, loader)
            np.save(f'fmri_data/{vid}_{model_type}_{tc}_{ee}_{ll}_allframes', frame_acts)

            #calculate components for pca
            n_comp = calc_pc_n(extract_pc(frame_acts),.95)
            
            #calculate final set of PCs
            pca2 = extract_pc(frame_acts, n_comp)
            model_pcs = pca2.transform(frame_acts) #reduce dimensionality of data using model PCs
            #plot pc variance explained
            
            #downsample to scale of fmri
            downsample_ts = down_sample(model_pcs)

            #convolve to hrf
            final_ts = convolve_hrf(model_pcs)
            np.save(f'{results_dir}/{vid}_{model_type}_{tc}_{ee}_{ll}_frame_TS', final_ts)
            print(tc, ee,ll, n_comp)

        
    
    
    

In [11]:
import seaborn as sns

#conv_ts = np.load(f'fmri_data/{model_type}_{train_cond[0]}_{layer}_TS.npy')

temp= np.transpose(conv_ts[:, 0:100])
print(temp.shape)

ax = plt.figure(figsize=(12, 12))
ax = sns.heatmap(temp)

ax.set(xlabel='Time (TR)', ylabel='Component')
plt.savefig(f'fmri_data/timeseries_example.png',bbox_inches='tight')

NameError: name 'conv_ts' is not defined